# Part 3 - LLM as Judge
- Use proprietary LLM to assess domain names created by fine tuned llm
- This finishes off the pipeline and utlizes a rubric to assess the names (in addition to the BLEU/ROUGE metrics we used already)

In [ ]:
!pip install -q openai

In [ ]:
import os
import openai
from openai import OpenAI
import json
import pandas as pd
from tqdm import tqdm

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
#openai.api_key = user_secrets.get_secret("openai_api_key")
api_key = user_secrets.get_secret("openai_api_key")

In [ ]:
def format_prompt(description: str, domain_name: dict) -> str:
    return f"""
You are an expert branding consultant. Your task is to evaluate the quality of a proposed domain name for a business.

Business Description:
\"\"\"{description}\"\"\"

Proposed Domain Name:
\"\"\"{domain_name}\"\"\"

Evaluate the domain on the following criteria:
1. **Relevance**: Does the domain name relate well to the business?
2. **Creativity**: Is the name original and imaginative?
3. **Brandability**: Is it catchy, easy to remember, and easy to spell?
4. **Safety**: Is the name appropriate and free of offensive language?

Provide a JSON response like:
{{
  "relevance": 4,
  "creativity": 5,
  "brandability": 4,
  "safety": 5,
  "comments": "Creative and brandable. Slightly generic but safe and relevant."
}}

ONLY respond with the JSON and nothing else.
"""

In [ ]:
# Initialize OpenAI client
client = OpenAI(api_key=api_key)

def call_judge(prompt):
    #prompt = format_prompt(description, domain_name)

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )

    content = response.choices[0].message.content.strip()
    try:
        parsed = json.loads(content)
    except json.JSONDecodeError:
        parsed = eval(content)  # fallback (not recommended)
        return None
    return parsed

In [ ]:
def evaluate_domains(pred_csv_path, output_path):
    df = pd.read_csv(pred_csv_path)
    evaluations = []

    for _, row in tqdm(df.iterrows(), total=len(df)):
        if not row['is_valid_domain']:
            continue
        prompt = format_prompt(row["description"], row["pred"])
        result = call_judge(prompt)

        if result:
            evaluations.append({**row, **result})
        else:
            evaluations.append({**row, "relevance": None, "creativity": None, 
                                "brandability": None, "safety": None, "comments": "Failed to evaluate"})

    pd.DataFrame(evaluations).to_csv(output_path, index=False)
    print(f"Saved evaluations to {output_path}")

In [ ]:
# Run the evaluations 
version = 'v1'
input_file = f'/kaggle/input/domain-name-generator/data/predictions_eval-{version}.csv'
output_file = f'judged_domains-{version}.csv'
evaluate_domains(input_file, output_file)